In [7]:
# Importing libraries
import pandas as pd

In [8]:
# Reading population data with age-group
populationDf = pd.read_excel('input/DDW-0000C-14.xls', names=['tableName','State Code','District','state/ut','agegroup','Persons','Males','Females','Persons_L2','Males_L2','Females_L2','Persons_L3','Males_L3','Females_L3'])
populationDf = populationDf[['State Code','state/ut','agegroup','Persons']]
populationDf = populationDf.iloc[6:, : ]
populationDf = populationDf.astype({'State Code':int})
populationDf = populationDf[populationDf.agegroup != '0-4']
populationDf.head(20)

,State Code,state/ut,agegroup,Persons
6,0,India,All ages,1210854977
8,0,India,5-9,126928126
9,0,India,10-14,132709212
10,0,India,15-19,120526449
11,0,India,20-24,111424222
12,0,India,25-29,101413965
13,0,India,30-34,88594951
14,0,India,35-39,85140684
15,0,India,40-44,72438112
16,0,India,45-49,62318327


In [9]:
pDf = pd.DataFrame()
# Grouping age-group so that they should be in same order as in language speakers data
for stCode in range(36):
    stDf = populationDf[populationDf['State Code'] == stCode]
    stDf = stDf.reset_index(drop=True)
    stDf.loc[6,'Persons'] = stDf[(stDf.agegroup == '30-34') | (stDf.agegroup == '35-39') | (stDf.agegroup == '40-44') | (stDf.agegroup == '45-49')]['Persons'].sum()
    stDf.loc[6,'agegroup'] = '30-49'
    stDf.loc[10,'Persons'] = stDf[(stDf.agegroup == '50-54') | (stDf.agegroup == '55-59') | (stDf.agegroup == '60-64') | (stDf.agegroup == '65-69')]['Persons'].sum()
    stDf.loc[10,'agegroup'] = '50-69'
    stDf.loc[14,'Persons'] = stDf[(stDf.agegroup == '70-74') | (stDf.agegroup == '75-79') | (stDf.agegroup == '80+')]['Persons'].sum()
    stDf.loc[14,'agegroup'] = '70+'
    stDf = stDf[(stDf.agegroup != 'All ages') & (stDf.agegroup != '35-39') & (stDf.agegroup != '40-44') & (stDf.agegroup != '45-49') & (stDf.agegroup != '55-59') & (stDf.agegroup != '60-64') & (stDf.agegroup != '65-69') & (stDf.agegroup != '75-79') & (stDf.agegroup != '80+')]
    pDf = pd.concat([pDf,stDf])

pDf

,State Code,state/ut,agegroup,Persons
1,0,India,5-9,126928126
2,0,India,10-14,132709212
3,0,India,15-19,120526449
4,0,India,20-24,111424222
5,0,India,25-29,101413965
...,...,...,...,...
5,35,State - ANDAMAN & NICOBAR ISLANDS (35),25-29,41722
6,35,State - ANDAMAN & NICOBAR ISLANDS (35),30-49,119161
10,35,State - ANDAMAN & NICOBAR ISLANDS (35),50-69,47692
14,35,State - ANDAMAN & NICOBAR ISLANDS (35),70+,8822


In [10]:
# Reading language speakers data with age-group
languageDf = pd.read_excel('input/DDW-C18-0000.xlsx', names=['State Code','District','state/ut','TRU','age-group','Persons_L2','Males_L2','Females_L2','Persons_L3','Males_L3','Females_L3'])
languageDf = languageDf[languageDf['TRU']=='Total'].drop(['TRU'], axis=1)
languageDf = languageDf[languageDf['age-group']!='Total'].drop(['District','Persons_L2','Males_L2','Females_L2','Males_L3','Females_L3'], axis=1).reset_index(drop=True)
languageDf = languageDf.astype({'State Code':int})
languageDf

,State Code,state/ut,age-group,Persons_L3
0,0,INDIA,5-9,1844108
1,0,INDIA,10-14,7254335
2,0,INDIA,15-19,12626717
3,0,INDIA,20-24,12834334
4,0,INDIA,25-29,10857171
...,...,...,...,...
319,35,ANDAMAN & NICOBAR ISLANDS,25-29,9716
320,35,ANDAMAN & NICOBAR ISLANDS,30-49,27080
321,35,ANDAMAN & NICOBAR ISLANDS,50-69,7803
322,35,ANDAMAN & NICOBAR ISLANDS,70+,771


In [11]:
ageIndiaDf = pd.DataFrame()
# Finding age-group from india and each state with highest percentage of people speaking three languages or more
for stCode in range(36):
    stDf = languageDf[languageDf['State Code'] == stCode].reset_index(drop=True)
    pStDf = pDf[pDf['State Code'] == stCode].reset_index(drop=True)
    stDf['percentage'] = (stDf['Persons_L3']/pStDf['Persons'])*100
    stDf = stDf.sort_values(by=['percentage'], ascending=False).reset_index(drop=True)
    ageIndiaDf = pd.concat([ageIndiaDf,stDf.head(1)])
ageIndiaDf = ageIndiaDf[['State Code','age-group','percentage']]
ageIndiaDf = ageIndiaDf.rename({'State Code':'state/ut'},axis=1)
ageIndiaDf

,state/ut,age-group,percentage
0,0,20-24,11.518442
0,1,20-24,29.580167
0,2,20-24,8.206521
0,3,15-19,45.763949
0,4,50-69,42.774903
0,5,20-24,3.094835
0,6,15-19,7.31862
0,7,70+,14.999231
0,8,15-19,2.884065
0,9,20-24,2.443396


In [6]:
# Generating output csv file from datframe
ageIndiaDf.to_csv('age-india.csv',index=False)